In [1]:
"""from datasets import load_dataset
import pandas as pd

scicite = load_dataset('hrithikpiyush/acl-arc')

# Convert to a pandas DataFrame
df = pd.DataFrame({
    'text': scicite['train']['text'],
    'label': scicite['train']['section_name'],
})

# Save to CSV
df.to_csv('aclarc.csv', index=False)"""

"from datasets import load_dataset\nimport pandas as pd\n\nscicite = load_dataset('hrithikpiyush/acl-arc')\n\n# Convert to a pandas DataFrame\ndf = pd.DataFrame({\n    'text': scicite['train']['text'],\n    'label': scicite['train']['section_name'],\n})\n\n# Save to CSV\ndf.to_csv('aclarc.csv', index=False)"

In [2]:
import pandas as pd

df = pd.read_csv('../datasets/aclarc.csv')
df.head(3)

,text,label
0,"Thus , over the past few years , along with ad...",introduction
1,"This was done by MERT optimization ( Och , 200...",experiments
2,"She evaluates 3,000 German verbs with a token ...",NaN


In [3]:
df.dropna(inplace=True)
df.head(3)

,text,label
0,"Thus , over the past few years , along with ad...",introduction
1,"This was done by MERT optimization ( Och , 200...",experiments
5,This is roughly an 11 % relative reduction in ...,conclusion


In [4]:
df.shape

(1223, 2)

In [5]:
"""
Now we do Data Preprocessing.
"""
import string, re, nltk
from string import punctuation
from nltk.tokenize import word_tokenize, RegexpTokenizer
from nltk.corpus import stopwords
from nltk.stem.porter import PorterStemmer
from nltk.stem import WordNetLemmatizer

regexp = RegexpTokenizer("[\w']+")

def convert_to_lowercase(text):
    return text.lower()
def remove_whitespace(text):
    return text.strip()
def remove_punctuation(text):
    punct_str = string.punctuation
    punct_str = punct_str.replace("'", "") 
    return text.translate(str.maketrans("", "", punct_str))
def remove_html(text):
    html = re.compile(r'<.*?>')
    return html.sub(r'', text)
def remove_http(text):
    http = "https?://\S+|www\.\S+" 
    pattern = r"({})".format(http) 
    return re.sub(pattern, "", text)
# Stopwords
stops = stopwords.words("english") 
addstops = ["among", "onto", "shall", "thrice", "thus", "twice", "unto", "us", "would"] 
allstops = stops + addstops
def remove_stopwords(text):
    return " ".join([word for word in regexp.tokenize(text) if word not in allstops])
stemmer = PorterStemmer()
def text_stemmer(text):
    text_stem = " ".join([stemmer.stem(word) for word in regexp.tokenize(text)])
    return text_stem
def discard_non_alpha(text):
    word_list_non_alpha = [word for word in regexp.tokenize(text) if word.isalpha()]
    text_non_alpha = " ".join(word_list_non_alpha)
    return text_non_alpha

In [6]:
def text_normalizer(text):
    text = convert_to_lowercase(text)
    text = remove_whitespace(text)
    text = re.sub('\n' , '', text) 
    text = re.sub('\[.*?\]', '', text) 
    text = remove_http(text)
    text = remove_punctuation(text)
    text = remove_html(text)
    text = remove_stopwords(text)
    text = discard_non_alpha(text)
    return text

In [7]:
df['text'] = df['text'].apply(text_normalizer)

In [8]:
df['text'][0]

'past years along advances use learning statistical methods acquisition full parsers collins charniak charniak ratnaparkhi significant progress made use statistical learning methods recognize shallow parsing patterns syntactic phrases words participate syntactic relationship church ramshaw marcus argamon et al cardie pierce munoz et al punyakanok roth buchholz et al tjong kim sang buchholz'

In [9]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split


vectorizer = TfidfVectorizer(strip_accents='unicode', analyzer='word', ngram_range=(1,2), max_features=15000, smooth_idf=True, sublinear_tf=True)
df_feature = vectorizer.fit_transform(df['text'])
df_features_train, df_features_test, df_target_train, df_target_test = train_test_split(df_feature, df['label'], test_size=0.2, random_state=42)

In [10]:
from sklearn.naive_bayes import MultinomialNB
nb_model = MultinomialNB()
nb_model.fit(df_features_train, df_target_train)
target_prediction = nb_model.predict(df_features_test)

In [11]:
from sklearn.metrics import classification_report, accuracy_score

print(classification_report(df_target_test, target_prediction))

              precision    recall  f1-score   support

  conclusion       1.00      0.11      0.19        19
 experiments       0.78      0.17      0.27        42
introduction       0.44      1.00      0.61        91
      method       0.00      0.00      0.00        19
related work       1.00      0.34      0.51        74

    accuracy                           0.51       245
   macro avg       0.64      0.32      0.32       245
weighted avg       0.67      0.51      0.44       245



/Users/makarwuckert/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/makarwuckert/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result))
/Users/makarwuckert/opt/anaconda3/lib/python3.9/site-packages/sklearn/metrics/_classification.py:1344: UndefinedMetricWarning: Precision and F-score are ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, msg_start, len(result)

In [12]:
accuracy_score = accuracy_score(df_target_test, target_prediction)

In [13]:
accuracy_score

0.5102040816326531